In [1]:
%pip install -U insightface onnxruntime-gpu  numpy

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 439.5/439.5 kB 32.8 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.6/121.6 MB 12.3 MB/s eta 0:00:0000:0100:01
  Using cached numpy-1.25.0-cp39-cp39-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (17.7 MB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 125.7/125.7 kB 27.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.6/14.6 MB 52.3 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.0/46.0 kB 10.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.7/5.7 MB 70.7 MB/s eta 0:00:00:00:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.1/49.1 MB 14.3 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.3/17.3 MB 19.1 MB/s eta 0:00:00

In [5]:
%pip install gdown moviepy ffmpeg insightface

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 388.3/388.3 kB 26.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 26.9/26.9 MB 26.5 MB/s eta 0:00:0000:0100:01
  Created wheel for moviepy: filename=moviepy-1.0.3-py3-none-any.whl size=110728 sha256=da130fc6325e068e580bc3b2d22f66cb1ea2676d6f660a5efe2d1b28730251a2
  Stored in directory: /root/.cache/pip/wheels/29/15/e4/4f790bec6acd51a00b67e8ee1394f0bc6e0135c315f8ff399a
  Created wheel for ffmpeg: filename=ffmpeg-1.4-py3-none-any.whl size=6082 sha256=815b64df945d9f6e6e70f8ebb07b90eb1ad3e68685fc0ab669d728b2791e5f30
  Stored in directory: /root/.cache/pip/wheels/1d/57/24/4eff6a03a9ea0e647568e8a5a0546cdf957e3cf005372c0245
Successfully built moviepy ffmpeg
  Attempting uninstall: decorator
    Found existing installation: decorator 5.1.1
    Uninstalling decorator-5.1.1:
      Successfully uninstalled decorator-5.1.1
Note: you may need to restart the 

In [27]:
import gdown
import cv2
import os
from moviepy.editor import VideoFileClip,AudioFileClip
import glob
import os.path as osp
import shutil
import insightface
from insightface.app import FaceAnalysis
from insightface.data import get_image as ins_get_image

In [28]:
assert insightface.__version__ >= '0.7'

# Define variables
drive_file_id = '1-16Qu0ekQda4Y3MEJ9IW9K7IFq7zZgWm'  # replace 'FILE_ID' with your file's ID
video_output = 'my_video.mp4'
frame_folder = '/notebooks/frame_folder'
swapped_folder = '/notebooks/swapped'


# Create the frame folder if it doesn't exist
if not os.path.exists(frame_folder):
    os.makedirs(frame_folder)
# Create the frame folder if it doesn't exist
if not os.path.exists(swapped_folder):
    os.makedirs(swapped_folder)

In [ ]:
shutil.rmtree(frame_folder)
shutil.rmtree(swapped_folder)

In [11]:
import requests

url = "http://example.com/path_to_your_file"  # replace with your file's URL
response = requests.get(url)

with open(video_output, 'wb') as f:  # replace with the path where you want to save the file
    f.write(response.content)

In [29]:
# Download video from Google Drive
url = f'https://drive.google.com/uc?id={drive_file_id}'
gdown.download(url, video_output, quiet=False)

Downloading...
From: https://drive.google.com/uc?id=1-16Qu0ekQda4Y3MEJ9IW9K7IFq7zZgWm
To: /notebooks/my_video.mp4
100%|██████████| 6.65M/6.65M [00:00<00:00, 42.8MB/s]


'my_video.mp4'

In [30]:

# Extract frames from video
vidcap = cv2.VideoCapture(video_output)
success, image = vidcap.read()
count = 0
while success:
    cv2.imwrite(os.path.join(frame_folder, f"frame{count}.jpg"), image)  # save frame as JPEG file
    success, image = vidcap.read()
    count += 1

In [31]:
# Your face swapping script
app = FaceAnalysis(name='buffalo_l')
app.prepare(ctx_id=0, det_size=(640, 640))
swapper = insightface.model_zoo.get_model('/notebooks/inswapper_128.onnx')

source_img = cv2.imread('/notebooks/DSC06729.JPG')
source_faces = app.get(source_img)
source_faces = sorted(source_faces, key=lambda x: x.bbox[0])
assert len(source_faces) > 0
source_face = source_faces[0]  # use the first face found in the source image

image_files = glob.glob(os.path.join(frame_folder, '*.jpg'))  # get all jpg files in the frame folder

# Process each image and save the result
for img_file in image_files:
    img = cv2.imread(img_file)
    faces = app.get(img)
    faces = sorted(faces, key=lambda x: x.bbox[0])
    res = img.copy()
    for face in faces:
        res = swapper.get(res, face, source_face, paste_back=True)
    cv2.imwrite(osp.join(swapped_folder, osp.basename(img_file)), res)



Applied providers: ['CUDAExecutionProvider', 'CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}, 'CUDAExecutionProvider': {'device_id': '0', 'gpu_mem_limit': '18446744073709551615', 'gpu_external_alloc': '0', 'gpu_external_free': '0', 'gpu_external_empty_cache': '0', 'cudnn_conv_algo_search': 'EXHAUSTIVE', 'cudnn_conv1d_pad_to_nc1d': '0', 'arena_extend_strategy': 'kNextPowerOfTwo', 'do_copy_in_default_stream': '1', 'enable_cuda_graph': '0', 'cudnn_conv_use_max_workspace': '1', 'tunable_op_enable': '0', 'enable_skip_layer_norm_strict_mode': '0', 'tunable_op_tuning_enable': '0'}}
find model: /root/.insightface/models/buffalo_l/1k3d68.onnx landmark_3d_68 ['None', 3, 192, 192] 0.0 1.0
Applied providers: ['CUDAExecutionProvider', 'CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}, 'CUDAExecutionProvider': {'device_id': '0', 'gpu_mem_limit': '18446744073709551615', 'gpu_external_alloc': '0', 'gpu_external_free': '0', 'gpu_external_empty_cache': '0', 'cudnn_con

In [34]:
import subprocess

# Get the processed images
processed_images = glob.glob(os.path.join(swapped_folder, '*.jpg'))

# Sort the processed images (this may be necessary depending on how your files are named)
processed_images.sort()

# Initialize the video writer
height, width, _ = cv2.imread(processed_images[0]).shape
print(height, width)

# Define the command
command = f'ffmpeg -y -r 30 -f image2 -s {width}x{height} -i {swapped_folder}/frame%01d.jpg -vcodec libx264 -crf 25 -pix_fmt yuv420p output.mp4'

# Execute the command
subprocess.call(command, shell=True)

1280 720


ffmpeg version 4.2.7-0ubuntu0.1 Copyright (c) 2000-2022 the FFmpeg developers
  built with gcc 9 (Ubuntu 9.4.0-1ubuntu1~20.04.1)
  configuration: --prefix=/usr --extra-version=0ubuntu0.1 --toolchain=hardened --libdir=/usr/lib/x86_64-linux-gnu --incdir=/usr/include/x86_64-linux-gnu --arch=amd64 --enable-gpl --disable-stripping --enable-avresample --disable-filter=resample --enable-avisynth --enable-gnutls --enable-ladspa --enable-libaom --enable-libass --enable-libbluray --enable-libbs2b --enable-libcaca --enable-libcdio --enable-libcodec2 --enable-libflite --enable-libfontconfig --enable-libfreetype --enable-libfribidi --enable-libgme --enable-libgsm --enable-libjack --enable-libmp3lame --enable-libmysofa --enable-libopenjpeg --enable-libopenmpt --enable-libopus --enable-libpulse --enable-librsvg --enable-librubberband --enable-libshine --enable-libsnappy --enable-libsoxr --enable-libspeex --enable-libssh --enable-libtheora --enable-libtwolame --enable-libvidstab --enable-libvorbis --e

0

In [38]:
# Extract audio from original video and add it to the new video
clip = VideoFileClip(video_output)
clip.audio.write_audiofile("temp_audio.mp3")

final_video = VideoFileClip("/notebooks/output.mp4")
final_video.audio = AudioFileClip("/notebooks/temp_audio.mp3")
final_video.write_videofile("/notebooks/output_with_audio.mp4")

MoviePy - Writing audio in temp_audio.mp3


MoviePy - Done.
Moviepy - Building video /notebooks/output_with_audio.mp4.
MoviePy - Writing audio in output_with_audioTEMP_MPY_wvf_snd.mp3


MoviePy - Done.
Moviepy - Writing video /notebooks/output_with_audio.mp4



TypeError: must be real number, not NoneType